<a href="https://colab.research.google.com/github/tjdrhks0808/Seminar_in_Statistics/blob/gh-pages/KcBERT_Base_Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Last Update @ 2020.12.04

- Huggingface Transformers 4.0.0  버전 반영

# 패키지 import & 기본 Args 설정

In [ ]:
!pip install -q transformers pytorch_lightning soynlp

     |████████████████████████████████| 2.6 MB 38.6 MB/s 
     |████████████████████████████████| 813 kB 44.3 MB/s 
     |████████████████████████████████| 416 kB 60.9 MB/s 
     |████████████████████████████████| 3.3 MB 63.0 MB/s 
     |████████████████████████████████| 636 kB 52.8 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 829 kB 57.9 MB/s 
     |████████████████████████████████| 118 kB 61.0 MB/s 
     |████████████████████████████████| 10.6 MB 59.6 MB/s 
     |████████████████████████████████| 234 kB 66.8 MB/s 
     |████████████████████████████████| 1.3 MB 66.2 MB/s 
     |████████████████████████████████| 294 kB 65.2 MB/s 
     |████████████████████████████████| 142 kB 61.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires tensorboard~=2.5, but you have tensorboard 2

In [ ]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import BertForSequenceClassification, BertTokenizer, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
# import emoji
from soynlp.normalizer import repeat_normalize

# Package 설치 & 데이터 받기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/데이터_정제.csv')
raw = pd.read_excel('/content/drive/MyDrive/NLP/공모전_제공_데이터(1차).xlsx')

In [ ]:
raw_segment = []
for i in range(1000):
  raw_segment.append(raw.iloc[:,-1][i][:2])

In [ ]:
set(raw_segment)

{'불만', '중립', '칭찬'}

In [ ]:
neg_idx=[]
pos_idx=[]
neut_idx=[]
disp_idx=[]
for i in range(1000):
  if raw_segment[i] == '불만':
    neg_idx.append(i)
  elif raw_segment[i] == '칭찬':
    pos_idx.append(i)
  elif raw_segment[i] == '중립':
    neut_idx.append(i)
  else: disp_idx.append(i)

In [ ]:
label = [0 for i in range(1000)]
for i in neg_idx:
  label[i] = 1

for i in neut_idx:
  label[i] = 2

for i in disp_idx:
  label[i] = 3

In [ ]:
df['label'] = label

In [ ]:
df = df[ ['네이버', 'label'] ]

In [ ]:
df[ df['label'] == 2 ]

,네이버,label
516,보이는 화면,2
517,화면으로 보는 디지털 ARS,2
518,화면 상담 서비스,2
519,터치식 화면,2
520,화면을 구현하는 ars,2
521,화면 연결,2
522,바로 보는 화면 서비스,2


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.25, random_state=42)

In [ ]:
train['label'].value_counts()

1    392
0    353
2      5
Name: label, dtype: int64

In [ ]:
test['label'].value_counts()

1    124
0    124
2      2
Name: label, dtype: int64

In [ ]:
cal_len = []
for i in range(1000):
  cal_len.append(len(df['네이버'][i]))
print(max(cal_len))

246


In [ ]:
pd.DataFrame(cal_len).describe()

,0
count,1000.000000
mean,26.110000
std,24.087509
min,2.000000
25%,12.000000
50%,18.000000
75%,31.000000
max,246.000000


# 기본 학습 Arguments

In [ ]:
class Arg:
    random_seed: int = 42  # Random Seed
    pretrained_model: str = 'beomi/kcbert-base'  # Transformers PLM name
    pretrained_tokenizer: str = ''  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    auto_batch_size: str = 'power'  # Let PyTorch Lightening find the best batch size 
    batch_size: int = 0  # Optional, Train/Eval Batch Size. Overrides `auto_batch_size` 
    lr: float = 5e-6  # Starting Learning Rate
    epochs: int = 20  # Max Epochs
    max_length: int = 100  # Max Length input size
    report_cycle: int = 100  # Report (Train Metrics) Cycle
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    tpu_cores: int = 0  # Enable TPU with 1 core or 8 cores

args = Arg()

## 기본값을 Override 하고싶은 경우 아래와 같이 수정

In [ ]:
!nvidia-smi

Tue Jul 27 14:05:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

위에서 GPU가 V100/P100이면 아래 `batch_size`  를 32 이상으로 하셔도 됩니다.

In [ ]:
# args.tpu_cores = 8  # Enables TPU
args.fp16 = True  # Enables GPU FP16
args.batch_size = 4  # Force setup batch_size

# Model 만들기 with Pytorch Lightning

In [ ]:
class Model(LightningModule):
    def __init__(self, options):
        super().__init__()
        self.args = options
        self.bert = BertForSequenceClassification.from_pretrained(self.args.pretrained_model, num_labels = 3)
        self.tokenizer = BertTokenizer.from_pretrained(
            self.args.pretrained_tokenizer
            if self.args.pretrained_tokenizer
            else self.args.pretrained_model
        )

    def forward(self, **kwargs):
        return self.bert(**kwargs)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits
        
        preds = logits.argmax(dim=-1)

        y_true = labels.cpu().numpy()
        y_pred = preds.cpu().numpy()

        # Acc, Precision, Recall, F1
        metrics = [
            accuracy_score(y_true=y_true, y_pred=y_pred),
            precision_score(y_true=y_true, y_pred=y_pred, average='micro'),
            recall_score(y_true=y_true, y_pred=y_pred, average='micro'),
            f1_score(y_true=y_true, y_pred=y_pred, average='micro')
        ]

        tensorboard_logs = {
            'train_loss': loss.cpu().detach().numpy().tolist(),
            'train_acc': metrics[0],
            'train_precision': metrics[1],
            'train_recall': metrics[2],
            'train_f1': metrics[3],
        }
        if (batch_idx % self.args.report_cycle) == 0:
            print()
            pprint(tensorboard_logs)
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def validation_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(outputs)

        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']

        # Acc, Precision, Recall, F1
        metrics = [
            accuracy_score(y_true=y_true, y_pred=y_pred),
            precision_score(y_true=y_true, y_pred=y_pred, average='micro'),
            recall_score(y_true=y_true, y_pred=y_pred, average='micro'),
            f1_score(y_true=y_true, y_pred=y_pred, average='micro')
        ]

        tensorboard_logs = {
            'val_loss': loss,
            'val_acc': metrics[0],
            'val_precision': metrics[0],
            'val_recall': metrics[1],
            'val_f1': metrics[2],
        }

        print()
        pprint(tensorboard_logs)
        return {'loss': _loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        if self.args.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.args.lr)
        elif self.args.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.args.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.args.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.args.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def preprocess_dataframe(self, df):

        df['네이버'] = df['네이버'].map(lambda x: self.tokenizer.encode(
            str(x),
            padding='max_length',
            max_length=self.args.max_length,
            truncation=True,
        ))
        return df

    def train_dataloader(self):
        df = train
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['네이버'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=True,
            num_workers=self.args.cpu_workers,
        )

    def val_dataloader(self):
        df = test
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['네이버'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )


In [ ]:
def main():
    print("Using PyTorch Ver", torch.__version__)
    print("Fix Seed:", args.random_seed)
    seed_everything(args.random_seed)
    model = Model(args)

    print(":: Start Training ::")
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        auto_scale_batch_size=args.auto_batch_size if args.auto_batch_size and not args.batch_size else False,
        # For GPU Setup
        deterministic=torch.cuda.is_available(),
        gpus=-1 if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32,
        # For TPU Setup
        # tpu_cores=args.tpu_cores if args.tpu_cores else None,
    )
    trainer.fit(model)

# 학습!

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
main()

Global seed set to 42


Using PyTorch Ver 1.9.0+cu102
Fix Seed: 42


Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

:: Start Training ::


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...


{'val_f1': 0.792396213898552,
 'val_loss': 0.4727214574813843,
 'val_precision': 0.7938121718841143,
 'val_recall': 0.796}

{'train_f1': 0.6458333333333333,
 'train_loss': 0.6779346466064453,
 'train_precision': 0.7362012987012987,
 'train_recall': 0.65625}



{'val_f1': 0.8047360000000001,
 'val_loss': 0.4655737578868866,
 'val_precision': 0.8015238095238095,
 'val_recall': 0.808}

{'train_f1': 0.8442118226600985,
 'train_loss': 0.4443778991699219,
 'train_precision': 0.8463235294117648,
 'train_recall': 0.84375}



{'val_f1': 0.7757926007694682,
 'val_loss': 0.48407885432243347,
 'val_precision': 0.7817090311986864,
 'val_recall': 0.78}

{'train_f1': 0.7991898148148148,
 'train_loss': 0.4664745330810547,
 'train_precision': 0.7896002024291497,
 'train_recall': 0.8125}



{'val_f1': 0.8043139083139084,
 'val_loss': 0.4650781750679016,
 'val_precision': 0.8064831168831168,
 'val_recall': 0.808}

{'train_f1': 0.75,
 'train_loss': 0.48464393615722656,
 'train_precision': 0.75,
 'train_recall': 0.75}



{'val_f1': 0.7957270955165692,
 'val_loss': 0.45923104882240295,
 'val_precision': 0.8029293993677555,
 'val_recall': 0.8}

